
# 07: Rule Ablation Study（理論駆動の改善比較）

このノートは **研究用途** です。実運用コードではありません。  
目的は、EMAクロス戦略の改善案を要素分解して比較することです。

比較する4バリアント:
- **A)** EMA cross only
- **B)** EMA cross + ATR regime（現行の考え方）
- **C)** EMA cross + trend strength filter（`|ema_diff| / ATR`）
- **D)** EMA cross + cooldown（シグナル後N日エントリー停止）



## 1) 事前知識（式）

- EMA差分: `ema_diff_t = EMA_fast_t - EMA_slow_t`
- クロス判定:
  - 上抜け: `ema_diff_{t-1} <= 0 and ema_diff_t > 0`
  - 下抜け: `ema_diff_{t-1} >= 0 and ema_diff_t < 0`
- リーク回避: 
  - シグナルは **t時点情報** で作る
  - 実現収益は **t→t+1** を使う（`next_day_return_t`）


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from quantlab.indicators import ema, atr
from quantlab.backtest import (
    generate_positions_from_signals,
    compute_strategy_returns,
    summarize_performance,
)

plt.style.use("seaborn-v0_8")
pd.set_option("display.float_format", lambda x: f"{x:,.5f}")


In [ ]:

# 研究設定（必要に応じて変更）
SYMBOL = "QQQ"
PERIOD = "8y"
EMA_FAST = 12
EMA_SLOW = 26
ATR_PERIOD = 14
REGIME_WIN = 60
TREND_STRENGTH_THRESHOLD = 0.15
COOLDOWN_DAYS = 5

# yfinanceはプロジェクト既存依存。
# インターネット接続がない環境では失敗するため、その場合はCSV等に置き換えてください。
df = yf.download(SYMBOL, period=PERIOD, auto_adjust=False, progress=False)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = [c[0] for c in df.columns]

df = df[["Open", "High", "Low", "Close", "Volume"]].dropna().copy()
print(df.head(3))
print(f"rows={len(df)}")


In [ ]:

# 特徴量計算（全バリアント共通）
work = df.copy()
work["ema_fast"] = ema(work["Close"], EMA_FAST)
work["ema_slow"] = ema(work["Close"], EMA_SLOW)
work["ema_diff"] = work["ema_fast"] - work["ema_slow"]
work["atr"] = atr(work, ATR_PERIOD)
work["atr_thresh"] = work["atr"].rolling(REGIME_WIN).median()
work["active_regime"] = work["atr"] > work["atr_thresh"]
work["trend_strength"] = (work["ema_diff"].abs() / work["atr"]).replace([np.inf, -np.inf], np.nan)


def base_cross_signal(frame: pd.DataFrame) -> pd.Series:
    """EMA crossのみで BUY / SELL / HOLD を返す。"""
    prev = frame["ema_diff"].shift(1)
    curr = frame["ema_diff"]
    buy = (prev <= 0) & (curr > 0)
    sell = (prev >= 0) & (curr < 0)

    signal = pd.Series("HOLD", index=frame.index, dtype="object")
    signal.loc[buy] = "BUY"
    signal.loc[sell] = "SELL"
    return signal


def apply_cooldown(signal: pd.Series, cooldown_days: int) -> pd.Series:
    """シグナル発生後cooldown_daysの間、新規シグナルをHOLDへ置換。"""
    out = signal.copy()
    remain = 0
    for i in range(len(out)):
        if remain > 0 and out.iat[i] in ("BUY", "SELL"):
            out.iat[i] = "HOLD"
        if out.iat[i] in ("BUY", "SELL"):
            remain = cooldown_days
        elif remain > 0:
            remain -= 1
    return out


In [ ]:

# バリアント定義
signals = {}

# A) EMA cross only
signals["A_EMA_ONLY"] = base_cross_signal(work)

# B) EMA cross + ATR regime
sig_b = base_cross_signal(work)
sig_b.loc[~work["active_regime"]] = "HOLD"
signals["B_EMA_ATR_REGIME"] = sig_b

# C) EMA cross + trend strength filter
sig_c = base_cross_signal(work)
strong = work["trend_strength"] >= TREND_STRENGTH_THRESHOLD
sig_c.loc[~strong] = "HOLD"
signals["C_EMA_TREND_STRENGTH"] = sig_c

# D) EMA cross + cooldown
signals["D_EMA_COOLDOWN"] = apply_cooldown(base_cross_signal(work), COOLDOWN_DAYS)


def evaluate_variant(frame: pd.DataFrame, signal: pd.Series) -> tuple[pd.Series, pd.Series, pd.Series]:
    local = frame.copy()
    local["signal"] = signal
    pos = generate_positions_from_signals(local)
    rets = compute_strategy_returns(local, pos)
    summary = summarize_performance(rets)

    # turnoverはポジション変化の絶対値平均で簡易化。
    summary["turnover"] = pos.diff().abs().fillna(0.0).mean()
    return pos, rets, summary


rows = []
ret_map = {}
pos_map = {}
for name, sig in signals.items():
    pos, r, s = evaluate_variant(work, sig)
    s.name = name
    rows.append(s)
    ret_map[name] = r
    pos_map[name] = pos

summary_df = pd.DataFrame(rows)
summary_df = summary_df[[
    "n", "mean", "std", "hit_rate", "avg_win", "avg_loss", "expectancy",
    "sharpe_like", "cum_return", "max_drawdown", "turnover"
]]
summary_df.sort_values("expectancy", ascending=False)


## 2) 数値結果（要約テーブル）

In [ ]:

print("=== Variant Summary ===")
print(summary_df.sort_values("expectancy", ascending=False).round(5))


## 3) 可視化（累積リターン比較）

In [ ]:

plt.figure(figsize=(11, 5))
for name, r in ret_map.items():
    eq = (1 + r.fillna(0.0)).cumprod()
    plt.plot(eq.index, eq.values, label=name)

plt.title(f"Ablation Equity Curves ({SYMBOL})")
plt.ylabel("Equity (start=1)")
plt.legend()
plt.tight_layout()
plt.show()



## 4) What to observe（観察ポイント）

- フィルタ追加で **turnover** が減っているか？（取引コスト耐性のヒント）
- **expectancy** が改善しているか？（勝率ではなく1トレードの質）
- **max_drawdown** が浅くなったか？（資金管理しやすさ）
- 最良バリアントが期間依存で入れ替わらないか？（過剰適合の確認）
